In [ ]:
import os

os.environ['POLARS_MAX_THREADS'] = '16'

import polars as pl
import geopandas as gpd


In [ ]:
gdf = gpd.read_parquet('data/adresser_geo.pq')
gdf_kom = gpd.read_parquet('data/kommune.pq')
gdf_kom = gdf_kom.to_crs(25832)

gdf = gpd.overlay(gdf, gdf_kom, how = 'intersection')

In [ ]:
gdf.to_parquet('data/adresser_geo_voronoi_parsed.pq')

In [ ]:
gdf = gpd.read_parquet('data/adresser_geo.pq')
gdf